# Registration of Scanned Maps and Species Hotspots onto World Map for Half Earth Project Using Computer Vision
The sample code uses Computer Vision techniques to automatically register scanned images of any part of the world onto the world map and hence overlay the species hotspots depicted in the scanned images onto the world map.

## Pre-requisites
OpenCV library: This sample requires installation of OpenCV library. Kindly install OpenCV using the following command:
pip install opencv-contrib-python

## Data Used
The data used for this sample are the scanned images from Half Earth Project and the corresponding species region masks.

## Defining the constants

In [2]:
# Path to the folder where the scanned map images are placed
IMAGES_TO_REGISTER_FOLDER = r"C:\Users\ati11038\Documents\workspace\data" \
                            r"\half_earth\extracted_samples\extracted_maps"

# Path to the folder where the species region mask images are placed
IMAGES_TO_REGISTER_SPECIES_MASKS_FOLDER = r"C:\Users\ati11038\Documents" \
                                         r"\workspace\data\half_earth\extracted_samples" \
                                          r"\extracted_masks"

# File path and filename of world image file 
WORLD_MAP_IMAGE_PATH = r"C:\Users\ati11038\Documents\workspace\data\half_earth\world_map"
WORLD_MAP_IMAGE_NAME = "map_without_rivers.tif"

# Path to the folder where the outputs need to appear
RESULTS_FOLDER = r"C:\\Users\\ati11038\\Documents\\workspace\\data\\half_earth\\results\\main" \
                 r"\\22_4_2020\\3"

# Minimum expected IOU of homography transform result with reference homography to qualify
# as improved homography
# (float: 0.0 - 1.0)
IOU_THRESHOLD_HOMOGRAPHY = 0.85

# Minimum expected spread of estimated control points over the scanned image
# (float: 0.0 - 1.0)
CONTOUR_RECT_AREA_RATIO_THRESHOLD = 0.40

# Maximum allowed distance of control points from the nearest contour point
# for control point refinement
# (int: 1 - 10) 
REFINEMENT_DISTANCE_THRESHOLD = 5

# Minimum threshold for segmenting mask from species region mask image
# (int: 1 - 255)
SPECIES_REGION_MASK_THRESHOLD = 50
SPECIES_REGION_MASK_VALUE = 255

# Upper and lower thresholds for homogenizing query image
HOMOGENIZE_UPPER_THRESHOLD = 245
HOMOGENIZE_LOWER_THRESHOLD = 180

# Boundary values for Latitude and Longitude corresponding to world map image
LONGITUDE_EXTREME_LEFT = -180
LATITUDE_EXTREME_TOP = 85
LONGITUDE_EXTREME_RIGHT = 180
LATITUDE_EXTREME_BOTTOM = -85

# Maximum number of control points needed 
MAX_CONTROL_POINTS = 8

## Defining helper functions

In [3]:
import cv2

In [4]:
from datetime import datetime

def find_current_timestamp():
    """
    Creates a string representing the current timestamp

    :returns: 
        String: Current timestamp in the format "%H_%M_%S_%f"
    """

    datetime_obj = datetime.now()
    timestamp_str = datetime_obj.strftime("%H_%M_%S_%f")
    return timestamp_str


In [6]:
import os
current_image_prefix = ""

def write_image_to_disk(image_to_write, image_suffix):
    """
    Saves image to disk

    :param image_to_write: Image which needs to be written to disk
    :param image_suffix: Suffix for the image name to be formed
    
    :returns: 
        None
    """
    image_dir = RESULTS_FOLDER + "\\" + current_image_prefix
    
    # Create directory if it does not exist
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    
    image_name = current_image_prefix + "_" + image_suffix
    image_path =  image_dir + "\\" + find_current_timestamp() + "_" + image_name + ".jpg"
    cv2.imwrite(image_path, image_to_write)


## Defining pre-processing functions

In [ ]:
def preprocess(input_img, kernel_size):
    """
    Create a binary mask and get rid of features such as rivers, discontinuities

    :param input_img: Input image to be registered on the world map
    :param kernel_size: Kernel size for morphological operation

    :return
    image (2D array) : Image without features such as rivers, discontinuities
    """
    
    input_img_gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)

    input_height = input_img_gray.shape[0]
    input_width = input_img_gray.shape[1]

    binary_mask = np.zeros((input_height, input_width), dtype=np.uint8)

    for y in range(0, input_height):
        for x in range(0, input_width):
            if input_img_gray[y][x] > 250:
                binary_mask[y][x] = 255

    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    binary_mask_closed = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

    #hp.write_image_to_disk(binary_mask_closed, "binary_mask_closed")
    #hp.show_image(binary_mask_closed, "binary_mask_closed")

    binary_mask_closed_eroded = cv2.erode(binary_mask_closed, kernel, iterations=1)
    #hp.write_image_to_disk(binary_mask_closed_eroded, "binary_mask_closed_eroded")
    #hp.show_image(binary_mask_closed_eroded, "binary_mask_closed_eroded")

    preprocessed_img = np.zeros((input_height, input_width), dtype=np.uint8)

    for y in range(0, input_height):
        for x in range(0, input_width):
            if binary_mask_closed_eroded[y][x] == 255:
                preprocessed_img[y][x] = 255
            else:
                #modified_input_img[y][x] = input_img[y][x]
                preprocessed_img[y][x] = 0

    #hp.write_image_to_disk(modified_input_img, "modified_input_img")
    #hp.show_image(modified_input_img, "modified_input_img")

    return preprocessed_img


In [12]:
def homogenize(input_img):
    """
    Converts color image to binary image with landmass depicted in white and
    other regions depicted in black, also filling the species region pixels

    :param input_img: Input image to be homogenized
    
    :returns: 
        Image(2D array): Homogenized Image
    """

    input_img_gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)

    homogenized_img = np.zeros((input_img_gray.shape[0], input_img_gray.shape[1]), dtype=np.uint8)

    input_img_height = input_img_gray.shape[0]
    input_img_width = input_img_gray.shape[1]

    for y in range(0, input_img_height):
        for x in range(0, input_img_width):
            if input_img_gray[y][x] >= HOMOGENIZE_UPPER_THRESHOLD:
                homogenized_img[y][x] = 255
            else:
                homogenized_img[y][x] = 0

    #write_image_to_disk(homogenized_img, "homogenized_img")
    return homogenized_img


In [13]:
def create_species_mask(mask_img):
    """
    Create binary mask for species region using the original color species region image

    :param mask_img: Color image of species mask
    
    :returns: 
        Image(2D array): Mask representing species region
    """

    mask_img_gray = cv2.cvtColor(mask_img, cv2.COLOR_BGR2GRAY)
    ret, mask_img = cv2.threshold(mask_img_gray,
                                  SPECIES_REGION_MASK_THRESHOLD,
                                  SPECIES_REGION_MASK_VALUE,
                                  cv2.THRESH_BINARY)

    #kernel = np.ones((5, 5), np.uint8)
    #dilated_species_mask_img = cv2.dilate(species_mask_img, kernel, iterations=1)

    #hp.write_image_to_disk(species_mask_img, "species_mask_img")
    #hp.write_image_to_disk(dilated_species_mask_img, "dilated_species_mask_img")
    #hp.show_image(species_mask_img, "species_mask_img")

    return mask_img


In [11]:
def fill_mask_pixels(input_img, mask):
    """
    Fills the mask pixels of image with white

    :param input_img: Color map image with species region
    :param mask: Binary mask image of species region
    
    :returns: 
        Image(2D array): Filled color image
    """

    input_img_copy = input_img.copy()

    mask_height = mask.shape[0]
    mask_width = mask.shape[1]

    for y in range(0, mask_height):
        for x in range(0, mask_width):
            if mask[y][x] == 255:
                input_img_copy[y][x][0] = 255
                input_img_copy[y][x][1] = 255
                input_img_copy[y][x][2] = 255

    #write_image_to_disk(input_img_copy, "modified_original_img_to_register")

    return input_img_copy


## Defining computation helper functions

In [16]:
def calculate_distance(point1_x, point1_y, point2_x, point2_y):
    """
    Find distance between two points
    point1_x: x coordinate of first point
    point1_y: y coordinate of first point
    point2_x: x coordinate of second point
    point2_y: y coordinate of second point

    :returns:
        Distance: Euclidean distance between two points
    """

    distance = math.sqrt((point1_x - point2_x) ** 2 + (point1_y - point2_y) ** 2)
    return distance


In [19]:
def calculate_diagonal_length(length, height):
    """
    Find the length of diagonal of a rectangle

    :param length: Length of rectangle
    :param height: Height of rectangle

    :return
    Length of the diagonal
    """

    length_diag = math.sqrt((length)** 2 + (height)** 2)

    return length_diag

In [18]:
def add_offset_to_points(input_contour, offset_x, offset_y):
    """
    Add offset to points on contour

    :param input_contour: Contour which needs to be processed
    :param offset_x: Offset which needs to be added in x co-ordinate
    :param offset_y: Offset which needs to be added in y co-ordinate

    :returns: 
        processed contour: Contour with offsetted points
    """

    for point in input_contour:
        point[0][0] += offset_x
        point[0][1] += offset_y

    return input_contour


In [20]:
def create_random_color_list(size):
    """
    Generate a list of specified size with random color values

    :param size: Size of the list which needs to be returned

    :returns:
        List of random color values
    """

    color_list = []

    for i in range(0, size):
        color_tuple = tuple(np.random.choice(range(256), size=3))
        color_tuple = tuple([int(x) for x in color_tuple])
        color_list.append(color_tuple)

    return color_list

In [21]:
def calculate_iou_binary_images(image1, image2):
    """
    Calculate iou using two binary images

    :param image1: First binary image
    :param image2: Second binary image

    :returns:
        iou(float)
    """

    intersection_img = cv2.bitwise_and(image1, image2)
    intersection_pixels = cv2.countNonZero(intersection_img)

    union_img = cv2.bitwise_or(image1, image2)
    union_pixels = cv2.countNonZero(union_img)

    iou = intersection_pixels / union_pixels

    return iou


## Defining core algorithm functions using OpenCV library

In [14]:
def find_contours(input_img):
    """
    Find contours in the input image
    :param input_img : Input image for which contours need to be computed
    
    :returns:
        List of contours : Computed contours for the input image
        
    """
    input_img_gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    
    ret, input_img_gray_thresh = cv2.threshold(input_img_gray, 127, 255, 0)
    #hp.write_image_to_disk(img_thresh, "thresholded_img")
    #hp.show_image(img_thresh, "thresholded_img")

    contours, hierarchy = cv2.findContours(input_img_gray_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #contour_img = np.zeros((img_thresh.shape[0], img_thresh.shape[1]), dtype=np.uint8)
    #contour_img = cv2.drawContours(contour_img, contours, -1, (255, 255, 255), 1)
    #hp.write_image_to_disk(contour_img, "img_thresh_contour_img")
    #hp.show_image(contour_img, "contour_img")

    return contours


In [15]:
def detect_template(template, world_map_preprocessed, world_map_color,
                    start_scale, end_scale, num_scales):
    """
    Performs template matching to locate the template in the world map

    :param template: Template image which needs to be registered on the world map
    :param world_map_color: Color image of world map image on which the 
                            registration needs to be performed
    :param world_map_preprocessed: Preprocessed image of world map
    
    :returns:
    Detected Bounding Box Co-ordinates: start_x, start_y, end_x, end_y
    """

    template_height = template.shape[0]
    template_width = template.shape[1]

    start_x = 0
    start_y = 0
    end_x = 0
    end_y = 0
    max_val = -1

    found = None
    #match_map_img = None

    # loop over the scales of the image
    for scale in np.linspace(start_scale, end_scale, num_scales)[::-1]:
        # resize the image according to the scale, and keep track
        # of the ratio of the resizing
        resized = imutils.resize(world_map_preprocessed,
                                 width=int(world_map_preprocessed.shape[1] * scale))
        
        scale_factor = world_map_preprocessed.shape[1] / float(resized.shape[1])

        # If the resized image is smaller than the template, then break
        # from the loop
        if resized.shape[0] < template_height or resized.shape[1] < template_width:
            break
        
        # Match the template in the resized image and store the result image
        result = cv2.matchTemplate(resized, template, cv2.TM_CCOEFF)
        
        # Find the maximum template match score and location in the result image
        (_, max_val, _, max_loc) = cv2.minMaxLoc(result)

        # If there is a new maximum correlation value, then update
        # the bookkeeping variable
        if found is None or max_val > found[0]:
            found = (max_val, max_loc, scale_factor)
            #match_map_img = result

            
    # Check if we have found a match
    if found is not None:
        # Unpack the bookkeeping variable and compute the (x, y) coordinates
        # of the bounding box based on the resized ratio
        (max_val, max_loc, scale_factor) = found
        (start_x, start_y) = (int(max_loc[0] * scale_factor), int(max_loc[1] * scale_factor))
        (end_x, end_y) = (int((max_loc[0] + template_width) * scale_factor),
                          int((max_loc[1] + template_height) * scale_factor))
    else:
        max_val = -1
        
    # draw a bounding box around the detected result and display the image
    # cv2.rectangle(world_map_img, (start_x, start_y), (end_x, end_y), (0, 0, 255), 2)
    # hp.write_image_to_disk(world_map_img, "detected_image")
    # hp.show_image(world_map_img, "detected_image")
    #world_map_img_resized = imutils.resize(world_map_img_edge_gray, width=int(world_map_img.shape[1] / scale_factor))
    #hp.write_image_to_disk(world_map_img_resized, "world_map_img_resized")
    # match_map_img = (match_map_img / max_val) * 255
    # hp.write_image_to_disk(match_map_img, "match_map_img")
    #hp.write_image_to_disk(world_map_img_clone, "world_map_img_detected_regions")

    return start_x, start_y, end_x, end_y, max_val


In [17]:
def find_nearest_contour_point(input_contour, pt_x, pt_y):
    """
    Find point on contour which is nearest to the given point
    
    :param input_contour: Contour on which points needs to be evaluated for nearness
    :param pt_x: X co-ordinate of point
    :param pt_y: Y co-ordinate of point
    
    :returns:
    x, y, index, distance: x-cordinate, y-cordinate, index, distance
    """

    distance = cv2.pointPolygonTest(contour, (pt_x, pt_y), True)

    min_distance = float("inf")
    min_index = -1
    min_x = -1
    min_y = -1

    for i, contour_pt in enumerate(contour):
        distance = find_distance(contour_pt[0][0], contour_pt[0][1], pt_x, pt_y)

        if distance < min_distance:
            min_distance = distance
            min_index = i
            min_x = contour_pt[0][0]
            min_y = contour_pt[0][1]

    return min_x, min_y, min_index, min_distance


In [22]:
def compute_homography_outlier_mask(control_pts):
    """
    Computes homography from mapped control points and outlier mask using RANSAC

    :param control_pts: List of control points in both query image and world map
                        co-ordinates

    :returns:
        Computed Homography matrix
        Mask array with flags indicating whether control points inlier or not
    """
    
    if len(control_pts_list) < 4:
        return None, None
    
    src_points = np.zeros((len(control_pts_list), 2), dtype=np.float32)
    dst_points = np.zeros((len(control_pts_list), 2), dtype=np.float32)

    for i, control_pt in enumerate(control_pts_list):
        src_points[i, 0] = control_pt[0]
        src_points[i, 1] = -1 * control_pt[1]
        dst_points[i, 0] = control_pt[5]
        dst_points[i, 1] = -1 * control_pt[6]

    homography_matrix, outlier_mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5)

    return homography_matrix, outlier_mask

## Looping over all images and executing the image registration pipeline

In [9]:
from pathlib import Path
import glob

# Create list of image files to register and list of species region masks
path_maps = Path(IMAGES_TO_REGISTER_FOLDER)
print("Path to maps : " + str(path_maps))

path_masks = Path(IMAGES_TO_REGISTER_SPECIES_MASKS_FOLDER)
print("Path to masks : " + str(path_masks))

# Create list of scanned image and species mask filepaths  
scanned_images_filepaths = [file_path for file_path in path_maps.glob('**/*.jpg')]
species_masks_filepaths = [file_path for file_path in path_masks.glob('**/*.jpg')]

# Sort the image filenames in alphabetical order
scanned_images_filepaths = sorted(scanned_images_filepaths)
species_masks_filepaths = sorted(species_masks_filepaths)

# Check if the number of images to register is same as number of species mask images
assert len(scanned_images_filepaths) == len(species_masks_filepaths)

print("Number of scanned images to process: " + str(len(scanned_images_filepaths)))

# Loop over all the images and species masks and execute the pipeline
for index in range(0, len(scanned_images_filepaths)):
    
    scanned_image_filepath = scanned_images_filepaths[index]
    species_mask_filepath = species_masks_filepaths[index]
    
    # Extract the filenames of scanned image and species mask
    scanned_image_filepath_split_list = str(scanned_image_filepath).split("\\")
    scanned_image_filename_prefix = scanned_image_filepath_split_list[-1].split(".")[0]
  
    species_mask_filepath_split_list = str(species_mask_filepath).split("\\")
    species_mask_filename_prefix = species_mask_filepath_split_list[-1].split(".")[0]
    
    print("Processing " + scanned_image_filename_prefix + " & " + species_mask_filename_prefix)

    # Update current image prefix
    current_image_prefix = scanned_image_filename_prefix
    
    register(scanned_images_filepaths[index], species_masks_filepaths[index])


Path to maps : C:\Users\ati11038\Documents\workspace\data\half_earth\extracted_samples\extracted_maps
Path to masks : C:\Users\ati11038\Documents\workspace\data\half_earth\extracted_samples\extracted_masks
Number of scanned images to process: 1587
Processing 7lomys_mirae & 7lomys_mirae_mask
Processing 7lomys_watsoni & 7lomys_watsoni_mask
Processing 7ylomys_bullaris & 7ylomys_bullaris_mask
Processing 7ylomys_fulviventer & 7ylomys_fulviventer_mask
Processing 7yphlomys_cinereus & 7yphlomys_cinereus_mask
Processing 7yphlomys_nanus & 7yphlomys_nanus_mask
Processing _arvicola & _arvicola_mask
Processing _leggadina & _leggadina_mask
Processing _lophuromys & _lophuromys_mask
Processing _pseudohydromys & _pseudohydromys_mask
Processing _reithrodontomys & _reithrodontomys_mask
Processing abrawayaomys_chebezi & abrawayaomys_chebezi_mask
Processing abrawayaomys_ruschii & abrawayaomys_ruschii_mask
Processing abrothrix_ & abrothrix__mask
Processing abrothrix_manni & abrothrix_manni_mask
Processing a